<a href="https://colab.research.google.com/github/MekdiYilma/HumanitarianCrisisPrediction/blob/main/notebooks/EDA_crisis_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [6]:
yemen_df = pd.read_csv('../data/yemen_batch1.csv', index_col=[0])
yemen_df.iloc[:2]

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,927505667,20200605,202006,2020,2020.4247,NaN,NaN,NaN,NaN,NaN,...,4,"Wadah, ?ajjah, Yemen",YM,YM22,32859.0,16.4224,43.4252,11086319,20200605000000,https://aawsat.com/home/article/2318236/%D8%A7...
1,927453145,20200604,202006,2020,2020.4219,NaN,NaN,NaN,NaN,NaN,...,1,Yemen,YM,YM,NaN,15.5000,47.5000,YM,20200604183000,https://hournews.net/news-101869.htm


In [7]:
yemen_df.columns

Index(['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat',
       'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type',
       'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
       'Actor2Geo_ADM2Code', 'Actor2Geo_Lat', 'Act

In [8]:
features = ['SQLDATE', 'MonthYear', 'Year', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 
       'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 
       'Actor1Type1Code',  'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 
       'Actor2Type1Code',  'IsRootEvent',
       'EventCode',  'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code',   'Actor2Geo_Type',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
        'ActionGeo_Type', 
       'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 
        'DATEADDED','SOURCEURL']

In [9]:
len(yemen_df.columns), len(features)

(61, 36)

In [10]:
yemen_df.loc[:,['EventCode', 'EventBaseCode', 'EventRootCode']]

,EventCode,EventBaseCode,EventRootCode
0,130,130,13
1,42,42,4
2,51,51,5
3,51,51,5
4,20,20,2
...,...,...,...
19995,195,195,19
19996,138,138,13
19997,190,190,19
19998,51,51,5


In [12]:
event_code_lookup = pd.read_csv('../data/',header=None, names=['Event'])
event_code_lookup.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/event_code_lookup.csv'

In [70]:
event_code_lookup

,Event
0,01 MAKE PUBLIC STATEMENT
1,"010 Make statement, not specified below"
2,011 Decline comment
3,012 Make pessimistic comment
4,013 Make optimistic comment
...,...
305,202 Engage in mass killings
306,203 Engage in ethnic cleansing
307,"204 Use weapons of mass destruction, not s..."
308,"2041 Use chemical, biological, or radiologi..."


In [67]:
event_code_lookup.iloc[52,:][0].split(' ', 1)[1][4:]

'Express intent to institute political reform, not specified below'

In [85]:
event_code_lookup['EventCode'] = event_code_lookup.Event.str.split(' ', n=1, expand=True)[0]
event_code_lookup['EventName'] = event_code_lookup.Event.str.split(' ', n=1, expand=True)[1].str.slice(start=4)

In [86]:
event_code_lookup.head()

,Event,EventCode,EventName
0,01 MAKE PUBLIC STATEMENT,01,MAKE PUBLIC STATEMENT
1,"010 Make statement, not specified below",010,"Make statement, not specified below"
2,011 Decline comment,011,Decline comment
3,012 Make pessimistic comment,012,Make pessimistic comment
4,013 Make optimistic comment,013,Make optimistic comment


In [107]:
event_code_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Event      310 non-null    object
 1   EventCode  310 non-null    object
 2   EventName  310 non-null    object
dtypes: object(3)
memory usage: 7.4+ KB


In [88]:
yemen = yemen_df[features]
yemen

,SQLDATE,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_ADM2Code,Actor2Geo_Type,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,ActionGeo_Type,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,DATEADDED,SOURCEURL
0,20200605,202006,2020,None,None,None,None,None,None,None,None,None,None,ERI,ERITREA,ERI,None,None,None,None,None,None,None,0,130,130,13,3,-4.4,2,1,2,-5.271318,0,None,None,None,1,ER,ER,None,4,YM,YM22,32859,20200605000000,https://aawsat.com/home/article/2318236/%D8%A7...
1,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,YEM,ADEN,YEM,None,None,None,None,None,None,None,1,042,042,04,1,1.9,20,1,20,0.000000,0,None,None,None,1,SA,SA,None,1,YM,YM,None,20200604183000,https://hournews.net/news-101869.htm
2,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,IRN,IRAN,IRN,None,None,None,None,None,None,None,1,051,051,05,1,3.4,4,1,4,-1.095890,0,None,None,None,1,SA,SA,None,4,YM,YM05,32968,20200604230000,https://www.al-madina.com/article/688409/%D9%8...
3,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,GBR,UNITED KINGDOM,GBR,None,None,None,None,None,None,None,1,051,051,05,1,3.4,10,1,10,-0.668151,0,None,None,None,1,YM,YM,None,1,YM,YM,None,20200604000000,https://www.okaz.com.sa/news/local/2027016
4,20200604,202006,2020,None,None,None,None,None,None,None,None,None,None,YEMGOV,YEMEN,YEM,None,None,None,None,GOV,None,None,1,020,020,02,1,3.0,10,1,10,2.397260,0,None,None,None,1,YM,YM,None,1,YM,YM,None,20200604000000,https://www.okaz.com.sa/news/local/2027007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,20200408,202004,2020,SAU,SAUDI ARABIA,SAU,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,190,190,19,4,-10.0,5,1,5,0.310559,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200408220000,https://english.alarabiya.net/en/coronavirus/2...
19996,20200408,202004,2020,MED,SPOKESMAN,None,None,None,None,None,MED,None,None,None,None,None,None,None,None,None,None,None,None,0,030,030,03,1,4.0,6,1,6,-5.747126,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200408224500,https://www.upi.com/Top_News/World-News/2020/0...
19997,20200409,202004,2020,YEM,YEMEN,YEM,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,190,190,19,4,-10.0,1,1,1,-2.445652,1,SA,SA,None,0,None,None,None,4,YM,YM16,32956,20200409014500,https://english.alarabiya.net/en/coronavirus/2...
19998,20200409,202004,2020,MED,SPOKESMAN,None,None,None,None,None,MED,None,None,None,None,None,None,None,None,None,None,None,None,1,010,010,01,1,0.0,1,1,1,-8.333333,1,SA,SA,None,0,None,None,None,1,YM,YM,None,20200409023000,https://www.dw.com/en/saudi-led-coalition-decl...


In [92]:
yemen.SQLDATE = pd.to_datetime(yemen.SQLDATE, format='%Y%m%d', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [105]:
yemen.DATEADDED = pd.to_datetime(yemen.DATEADDED,  format='%Y%m%d%H%M%S', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [1]:
yemen.info()

NameError: ignored